Initialization

In [1]:
import pandas as pd
import numpy as np

#for data analysis
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from plotly.subplots import make_subplots

#for market-basket
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import plotly.graph_objects as go
from collections import Counter

Data Cleaning

In [2]:
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/bakery_sales_revised.csv')
df

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,10/30/2016 9:58,morning,weekend
1,2,Scandinavian,10/30/2016 10:05,morning,weekend
2,2,Scandinavian,10/30/2016 10:05,morning,weekend
3,3,Hot chocolate,10/30/2016 10:07,morning,weekend
4,3,Jam,10/30/2016 10:07,morning,weekend
...,...,...,...,...,...
20502,9682,Coffee,4/9/2017 14:32,afternoon,weekend
20503,9682,Tea,4/9/2017 14:32,afternoon,weekend
20504,9683,Coffee,4/9/2017 14:57,afternoon,weekend
20505,9683,Pastry,4/9/2017 14:57,afternoon,weekend


In [3]:
#small EDA
#check missing values
df.isnull().sum()
df.isnull().sum() / len(df)*100

#re-defining columns
def preprocess_data(df):

    #separate date_time to two separate columns
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['time'] = df['date_time'].dt.strftime('%H:%M')
    df['date'] = df['date_time'].dt.date

    #apply strip in case of extra spaces
    df = df.apply(lambda col: col.astype(str).str.strip() if col.dtype == 'object' else col)
    df['time'] = pd.to_datetime(df['time'])
    df['date'] = pd.to_datetime(df['date'])
    return df
    

#correlation analysis
def corr_data(df):

    #defining features for matrix
    temp_df = df.copy()
    temp_df = temp_df.drop(columns=['date_time'])
    for col in temp_df.select_dtypes(include=['object']).columns:
        temp_df[col] = LabelEncoder().fit_transform(temp_df[col].astype(str))

    #using spearman for non-linear relationship between features
    corr_matrix = temp_df.corr(method='spearman')
    
    #plotting graph
    fig = px.imshow(corr_matrix,x=corr_matrix.columns,y=corr_matrix.index,text_auto=True,color_continuous_scale='viridis',
                    aspect='auto',title='Correlation Analysis')
    fig.show()

df = preprocess_data(df)
corr_data(df)

 


C:\Users\josep\AppData\Local\Temp\ipykernel_2856\223757292.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])


In [4]:
#top 5 products by no of order
def product_order(df):
    temp_df = pd.DataFrame()
    temp_df[['transaction_id','item']] = df[['Transaction','Item']]

    #get top 5 products 
    temp_df = temp_df.groupby('item')['transaction_id'].count().nlargest(5).reset_index(name='count')

    #plotting graph 
    fig = px.bar(temp_df,x='count',y='item',barmode='group',title='Top 5 Product by No.of Orders',
                 labels={'item':'Item','count':'No.of Order'})
    fig.show()

def day_order(df):
    temp_df = pd.DataFrame()
    temp_df [['transaction_id','item','period_day','period_week']] = df[['Transaction','Item','period_day','weekday_weekend']]

    #get individual hours and day names
    temp_df['hour'] = df['time'].dt.hour
    temp_df['day_name'] = df['date'].dt.day_name()
    
    temp_df1 = temp_df.groupby(['period_day','hour'])['transaction_id'].count().reset_index(name='count')
    temp_df2 = temp_df.groupby(['period_week','day_name'])['transaction_id'].count().reset_index(name='count')
    
    #plotting graphs
    fig = make_subplots(rows=1,cols=2,subplot_titles=('Order Distribution by Period of Day','Order Distribution by Day in Week'),
                        specs=[[{'type':'sunburst'},{'type':'sunburst'}]])

    fig1 = px.sunburst(temp_df1,path=['period_day','hour'],values='count',color='period_day', color_discrete_sequence=px.colors.qualitative.Plotly)
    fig1.update_traces(textinfo='label+percent entry')

    fig2 = px.sunburst(temp_df2,path=['period_week','day_name'],values='count',color='period_week', color_discrete_sequence=px.colors.qualitative.Plotly)
    fig2.update_traces(textinfo='label+percent entry')

    for trace in fig1.data:
        fig.add_trace(trace, row=1,col=1)
    for trace in fig2.data:
        fig.add_trace(trace, row=1,col=2)
    fig.show()

def year_month_order(df):
    temp_df = pd.DataFrame()
    temp_df [['transaction_id','item','date']] = df[['Transaction','Item','date']]

    #get individual months and year
    temp_df['year'] = df['date'].dt.year
    temp_df['month_no'] = df['date'].dt.month
    temp_df['month'] = df['date'].dt.month_name()

    temp_df1 = temp_df.groupby(['month_no','month'])['transaction_id'].count().reset_index(name='count')
    temp_df1 = temp_df1.sort_values(by='month_no',ascending=True)
    temp_df2 = temp_df.groupby('year')['transaction_id'].count().reset_index(name='count')

    #plotting graphs
    fig = make_subplots(rows=2,cols=1,subplot_titles=('No.of Orders by Month', 'No.of Orders by Year'))
    fig1 = px.line(temp_df1,x='month',y='count',markers=True)
    fig2 = px.bar(temp_df2,x='count',y='year',orientation='h')

    for trace in fig1.data:
        fig.add_trace(trace,row=1,col=1)
    for trace in fig2.data:
        fig.add_trace(trace,row=2,col=1)

    fig.update_xaxes(title_text="Month",row=1, col=1)
    fig.update_xaxes(title_text="No.of Feedback", row=2, col=1)
    fig.update_yaxes(title_text="Year", type='category', row=2, col=1)
    fig.update_yaxes(title_text="No.of Feedback", row=1, col=1)
    fig.show()


#product_order(df)
#day_order(df)
year_month_order(df)


Market-Basket Analysis Section

In [5]:
#create basket with unique transaction
basket = pd.crosstab(df['Transaction'], df['Item'])

#convert counts to 1/0
basket = basket.applymap(lambda x: 1 if x > 0 else 0)
basket


C:\Users\josep\AppData\Local\Temp\ipykernel_2856\2434247690.py:5: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#getting product combinations and likelihood of combination 
basket = basket.astype(bool)
frequent_items = apriori(basket,min_support=0.01,use_colnames=True)

rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.sort_values(by='lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
39,"(Tea, Coffee)",(Cake),0.049868,0.103856,0.010037,0.201271,1.937977,1.0,0.004858,1.121962,0.509401,0.069853,0.108705,0.148957
40,(Cake),"(Tea, Coffee)",0.103856,0.049868,0.010037,0.096643,1.937977,1.0,0.004858,1.051779,0.540090,0.069853,0.049230,0.148957
9,(Cake),(Hot chocolate),0.103856,0.058320,0.011410,0.109868,1.883874,1.0,0.005354,1.057910,0.523553,0.075683,0.054740,0.152760
8,(Hot chocolate),(Cake),0.058320,0.103856,0.011410,0.195652,1.883874,1.0,0.005354,1.114125,0.498236,0.075683,0.102434,0.152760
11,(Tea),(Cake),0.142631,0.103856,0.023772,0.166667,1.604781,1.0,0.008959,1.075372,0.439556,0.106736,0.070090,0.197779
10,(Cake),(Tea),0.103856,0.142631,0.023772,0.228891,1.604781,1.0,0.008959,1.111865,0.420538,0.106736,0.100611,0.197779
31,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431,1.0,0.007593,1.764582,0.332006,0.048464,0.433293,0.376936
30,(Coffee),(Toast),0.478394,0.033597,0.023666,0.049470,1.472431,1.0,0.007593,1.016699,0.615122,0.048464,0.016424,0.376936
36,"(Coffee, Bread)",(Pastry),0.090016,0.086107,0.011199,0.124413,1.444872,1.0,0.003448,1.043749,0.338354,0.067905,0.041916,0.127237
37,(Pastry),"(Coffee, Bread)",0.086107,0.090016,0.011199,0.130061,1.444872,1.0,0.003448,1.046033,0.336907,0.067905,0.044007,0.127237


In [7]:
'''
def offset_edge(x0, y0, x1, y1, offset=0.02):
    dx = x1 - x0
    dy = y1 - y0
    length = np.sqrt(dx**2 + dy**2)
    if length == 0:
        return x0, y0, x1, y1
    perp_dx = -dy / length * offset
    perp_dy = dx / length * offset
    return x0 + perp_dx, y0 + perp_dy, x1 + perp_dx, y1 + perp_dy
'''


#simple networkX graph
rules = rules.reset_index(drop=True)
G = nx.DiGraph()

#getting edges for each relation
for _,row in rules.iterrows():
    for a in row['antecedents']:
        for c in row['consequents']:
            G.add_edge(a,c,lift=row['lift'], confidence=row['confidence'])

pos = nx.spring_layout(G,k=0.8,seed=42)

#plotting edge
strong_x, strong_y = [], []
weak_x, weak_y= [], []

for u, v, edge in G.edges(data=True):
    x0,y0 = pos[u]
    x1,y1 = pos[v]

    #if G.has_edge(v, u) and u < v:  
        #x0, y0, x1, y1 = offset_edge(x0, y0, x1, y1, offset=0.03)

    if edge['lift'] >= 1.5:
        strong_x += [x0, x1, None]
        strong_y += [y0, y1, None]
    else:
        weak_x += [x0, x1, None]
        weak_y += [y0, y1, None]

strong_edge_trace = go.Scatter(
    x=strong_x, y=strong_y, line=dict(width=2,dash='dot',color="#10F014"), hoverinfo=None, mode='lines'
)
weak_edge_trace = go.Scatter(
    x=weak_x, y=weak_y, line=dict(width=1,dash='dot',color="#D81010"), hoverinfo=None, mode='lines'
)


#plotting node
node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x,y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

#define sizes of nodes (10 to 30)
ante_freq = Counter()
for ants in rules['antecedents']:
    for a in ants:
        ante_freq[a] += 1

node_sizes = []
for node in G.nodes():
    freq = ante_freq.get(node, 0)
    size = 10 + (freq / max(ante_freq.values())) * 20
    node_sizes.append(size)

node_trace = go.Scatter(
    x=node_x,y=node_y,mode='markers+text',text=node_text, textposition='top center', marker=dict(size=node_sizes,color="#1E1AE6",opacity=1), 
    textfont_weight='bold', hoverinfo='text'
)

#plotting with Plotly
fig = go.Figure(data=[strong_edge_trace,weak_edge_trace,node_trace])

fig.update_layout(title='Market Basket Analaysis Network Graph',title_x=0.5,showlegend=False,hovermode='closest')
fig.show()



In [8]:
#relationship between ruling metrics
fig = make_subplots(rows=2,cols=2)

fig1 = px.scatter(rules,x='support',y='confidence',hover_data=['lift'],color='antecedents')
fig1.update_traces(marker={'size': 10})
fig2 = px.scatter(rules,x='support',y='lift',hover_data=['confidence'],color='antecedents')
fig2.update_traces(marker={'size': 10})
fig3 = px.scatter(rules,x='confidence',y='lift',color='antecedents')
fig3.update_traces(marker={'size': 10})
fig4 = px.scatter(rules,x='antecedent support',y='consequent support',color='antecedents')
fig4.update_traces(marker={'size': 10})

for trace in fig1.data:
        fig.add_trace(trace,row=1,col=1)
for trace in fig2.data:
        fig.add_trace(trace,row=1,col=2)   
for trace in fig3.data:
        fig.add_trace(trace,row=2,col=1) 
for trace in fig4.data:
        fig.add_trace(trace,row=2,col=2)

fig.update_xaxes(title_text="Support", row=1, col=1)
fig.update_yaxes(title_text="Confidence", row=1, col=1)

fig.update_xaxes(title_text="Support", row=1, col=2)
fig.update_yaxes(title_text="Lift", row=1, col=2)

fig.update_xaxes(title_text="Confidence", row=2, col=1)
fig.update_yaxes(title_text="Lift", row=2, col=1)

fig.update_xaxes(title_text="Antecedent Support", row=2, col=2)
fig.update_yaxes(title_text="Consequent Support", row=2, col=2)

fig.update_layout(height=700,showlegend=False,title='Relationship between Ruling Metrics',title_x=0.5)
fig.show()



In [9]:
#heatmap for antecedents and consequents
#separate rows with multiple antecedents and consequents
heatmap_data = rules.explode('antecedents').explode('consequents')

#create pivot table for lift value
heatmap_matrix = heatmap_data.pivot_table(index='antecedents', columns='consequents', values='lift', fill_value=0)

fig = px.imshow(heatmap_matrix, x=heatmap_matrix.columns, y=heatmap_matrix.index, color_continuous_scale='Viridis', text_auto=True,                     
    labels=dict(x="Consequent", y="Antecedent", color="Lift"), aspect="auto")

fig.update_layout(title="Heatmap of Rules (Lift)", title_x=0.5)
fig.show()


In [10]:
#heatmap for antecedents and consequents
#separate rows with multiple antecedents and consequents
heatmap_data = rules.explode('antecedents').explode('consequents')

#create pivot table for lift value
heatmap_matrix = heatmap_data.pivot_table(index='antecedents', columns='consequents', values='confidence', fill_value=0)

fig = px.imshow(heatmap_matrix, x=heatmap_matrix.columns, y=heatmap_matrix.index, color_continuous_scale='Viridis', text_auto=True,                     
    labels=dict(x="Consequent", y="Antecedent", color="Confidence"), aspect="auto")

fig.update_layout(title="Heatmap of Rules (Confidence)", title_x=0.5)
fig.show()